In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import os
import random
import cv2
import time
from tempfile import TemporaryFile
get_ipython().magic('matplotlib inline')

In [2]:
def rotate(x):
    if random.random() < 0.5:
        x=np.fliplr(x)
    else:
        x=x
    
    return x

def get_train_data(batch_size,img_dir,labels_file):

  filenames = os.listdir(img_dir) #list of images in folder
  labels_df=pd.read_csv(open(labels_file,'rU'), encoding='utf-8', engine='c') #reads the csv file which contains labels/reggression values


  train_set=filenames[2000:] #all except first 2000 will be used for training

  batch_img=[]
  batch_labels=[]
  while True:
      random.shuffle(train_set) #shuffles the dataset

      for filename in train_set:

          if not filename.__contains__("png"):        #MaxOSX adds a .DS_STORE metadata file in every folder, this is for skipping that
              continue

          img=rotate(cv2.imread(os.path.join(img_dir,filename),0))

          label = np.around(labels_df[labels_df['images'] == filename]['sa1000'].values[0],decimals=3)

          batch_img.append(img)
          batch_labels.append(label)

          if len(batch_img) == batch_size:

              yield np.expand_dims(np.array(batch_img),axis=-1), np.expand_dims(np.array(batch_labels),axis=-1)

              batch_img = []
              batch_labels=[]
              
def get_test_data(batch_size,img_dir,labels_file):
  filenames = os.listdir(img_dir) 
  labels_df=pd.read_csv(open(labels_file,'rU'), encoding='utf-8', engine='c')
  test_set=filenames[:2000]

  batch_img=[]
  batch_labels=[]
  while True:

      random.shuffle(test_set)

      for filename in test_set:

          if not filename.__contains__("png"):        #MaxOSX adds a .DS_STORE metadata file in every folder, this is for skipping that
              continue

          img=rotate(cv2.imread(os.path.join(img_dir,filename),0))
          label = np.around(labels_df[labels_df['images'] == filename]['sa1000'].values[0],decimals=3)

          batch_img.append(img)
          batch_labels.append(label)

          if len(batch_img) == batch_size:

              yield np.expand_dims(np.array(batch_img),axis=-1), np.expand_dims(np.array(batch_labels),axis=-1)
              batch_img = []
              batch_labels=[]

In [3]:
def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1,1,1,1], padding='SAME')

def maxpool2d(x):
    # size of window   movement of window
    return tf.nn.max_pool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')

def averagepool(x):
    # size of window   movement of window
    return tf.nn.averagepool(x, ksize=[1,2,2,1], strides=[1,2,2,1], padding='SAME')


def l2_loss(gt, logits, thresh=1):
    print(gt.get_shape() ,logits.get_shape())
    square_loss = (tf.squeeze(gt) - tf.squeeze(logits)) ** 2

    return tf.reduce_mean(square_loss, axis=-1)
 
    
def l1_loss(gt, logits, thresh=1):
    print(gt.get_shape() ,logits.get_shape())
    loss = tf.abs(tf.squeeze(gt) - tf.squeeze(logits))

    return tf.reduce_mean(loss, axis=-1)
  
  #Commonly Used Loss Function https://en.wikipedia.org/wiki/Huber_loss
def smooth_l1_loss(gt, logits,sigma=0.5):
    gt=tf.squeeze(gt)
    logits=tf.squeeze(logits)
    with tf.variable_scope("L1_Loss"):
        absolute_loss = tf.abs(gt - logits)
        square_loss = 0.5 * (gt - logits) ** 2
        l1_loss = tf.where(tf.less(absolute_loss, sigma), square_loss, absolute_loss - sigma/2)

    return tf.reduce_mean(l1_loss, axis=-1)
  
def cnn1(x, TR_BIT = True):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,1,32],dtype=tf.float32, stddev=0.1)),
               'W_fc':tf.Variable(tf.random_normal([4096*32,1024],dtype=tf.float32, stddev=0.1)),
               'out':tf.Variable(tf.random_normal([1024, 1],dtype=tf.float32, stddev=0.1))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([1]))}

    #x = tf.reshape(x, shape=[None, 200, 200, 1])
    print(x.get_shape())

    bn1 = tf.layers.batch_normalization(conv2d(x, weights['W_conv1']) + biases['b_conv1'],training= TR_BIT)
    conv1 = tf.nn.sigmoid(bn1)
    conv1 = maxpool2d(conv1)
    print(conv1.get_shape())

    fc = tf.reshape(conv1,[-1,4096*32])
    fc = tf.nn.sigmoid(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, 0.8)

    output = tf.matmul(fc, weights['out'])+biases['out']
    print(output.get_shape())
    return output


def convolutional_neural_network1(x, TR_BIT = True):
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,1,32],dtype=tf.float32, stddev=0.1)),
               'W_conv2':tf.Variable(tf.random_normal([3,3,32,32],dtype=tf.float32, stddev=0.1)),
               'W_conv3':tf.Variable(tf.random_normal([3,3,32,64],dtype=tf.float32, stddev=0.1)),
               'W_conv4':tf.Variable(tf.random_normal([3,3,64,128],dtype=tf.float32, stddev=0.1)),
               'W_conv5':tf.Variable(tf.random_normal([3,3,128,256],dtype=tf.float32, stddev=0.1)),
               'W_fc':tf.Variable(tf.random_normal([16*256,1024],dtype=tf.float32, stddev=0.1)),
               'out':tf.Variable(tf.random_normal([1024, 1],dtype=tf.float32, stddev=0.1))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([32])),
               'b_conv3':tf.Variable(tf.random_normal([64])),
               'b_conv4':tf.Variable(tf.random_normal([128])),
               'b_conv5':tf.Variable(tf.random_normal([256])),

               'b_fc':tf.Variable(tf.random_normal([1024])),
               'out':tf.Variable(tf.random_normal([1]))}

    #x = tf.reshape(x, shape=[None, 200, 200, 1])
    print(x.get_shape())

    bn1 = tf.layers.batch_normalization(conv2d(x, weights['W_conv1']) + biases['b_conv1'],training= TR_BIT)
    conv1 = tf.nn.relu(bn1)
    conv1 = maxpool2d(conv1)
    print(conv1.get_shape())

    bn2  = tf.layers.batch_normalization(conv2d(conv1, weights['W_conv2']) + biases['b_conv2'],training= TR_BIT)
    conv2 = tf.nn.relu(bn2)
    conv2 = maxpool2d(conv2)
    print(conv2.get_shape())
    
    bn3  = tf.layers.batch_normalization(conv2d(conv2, weights['W_conv3']) + biases['b_conv3'],training= TR_BIT)
    conv3 = tf.nn.relu(bn3)
    conv3 = maxpool2d(conv3)
    print(conv3.get_shape())
    
    bn4 = tf.layers.batch_normalization(conv2d(conv3, weights['W_conv4']) + biases['b_conv4'],training= TR_BIT)
    conv4 = tf.nn.relu(bn4)
    conv4 = maxpool2d(conv4)
    print(conv4.get_shape())

    bn5 = tf.layers.batch_normalization(conv2d(conv4, weights['W_conv5']) + biases['b_conv5'],training= TR_BIT)
    conv5 = tf.nn.relu(bn5)
    conv5 = maxpool2d(conv5)
    print(conv5.get_shape())


    fc = tf.reshape(conv5,[-1,16*256])
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc'])+biases['b_fc'])
    fc = tf.nn.dropout(fc, 0.8)

    output = tf.matmul(fc, weights['out'])+biases['out']
#     output= tf.nn.sigmoid(output)
    print(output.get_shape())
    return output

In [5]:
img_dir= r'C:\Users\Peyman\Desktop\Naif\Journal\images'
labels_file= r'C:\Users\Peyman\Desktop\Naif\Journal\lgg.csv'
ckpt_dir= r'C:\Users\Peyman\Desktop\Naif\Journal\checkpoint'
load=0
batch_size = 256
number_files=len(os.listdir(r'C:\Users\Peyman\Desktop\Naif\Journal\images'))
next_train_batch=get_train_data(batch_size,img_dir,labels_file)
next_test_batch=get_test_data(batch_size,img_dir,labels_file)
name='_jgsa1000'

In [6]:
def train(): 
#   outfile = TemporaryFile()
  config = tf.ConfigProto()
  config.gpu_options.allow_growth = True
#   config = tf.ConfigProto(
#         device_count = {'GPU': 0}
#     )
  
  images = tf.placeholder(dtype=tf.float32, shape=[None, 128,128, 1], name="inp_batch")
  y = tf.placeholder(dtype=tf.float32, shape=[None,1], name="inp_batch")

  logits=convolutional_neural_network1(images)
  #logits=vgg(images)
  timing=[]
  

  loss = smooth_l1_loss(y,logits)
  #loss = l2_loss(y, logits)
  optimizer = tf.train.AdamOptimizer(1e-5).minimize(loss)

  #for saving a checkpoint after 50 epochs
  saver = tf.train.Saver(max_to_keep = 100)

  hm_epochs = 300
  with tf.Session(config=config) as sess:
      sess.run(tf.global_variables_initializer())
      starting_epoch=1
      if load > 0:
          print("Restoring", load, ".ckpt.....")
          saver.restore(sess, os.path.join(ckpt_dir,"backup"+ str(name)+ str(load)))
          starting_epoch=load+1

      for epoch in range(starting_epoch,hm_epochs):
  
          epoch_loss = 0

          for i in range(int((number_files-2000)/batch_size)):

            start=time.time()
            batch_x, batch_y = next(next_train_batch)
            _, step_loss = sess.run([optimizer, loss], feed_dict={images: batch_x, y: batch_y})
            epoch_loss += step_loss
            end= time.time()
            timing.append(end-start)

            print('#Epoch {} with total epochs {} at step {} loss: {} running average of batch loss {}, time {}'.format(epoch,hm_epochs,i, step_loss, epoch_loss/(i+1), time.time()-start))    
            with open('losses'+str(name)+'.txt','a') as loss_file:
            	loss_file.write('#Epoch {} with total epochs {} at step {} loss: {} running average of batch loss {}\n'.format(epoch,hm_epochs,i, step_loss, epoch_loss/(i+1)))
          
    
          if epoch%20==0:
        

              error = []
              gt=[]
              predic=[]

              for i in range(int(2000/batch_size)):
                    batch_x, batch_y = next(next_test_batch)
                    pred = sess.run([logits], feed_dict={images: batch_x})
                    error.append(np.mean(abs(pred-batch_y)))
                    predic.append(pred)
                    gt.append(batch_y)

              print( "avg difference between predicted and ground truth batch wise {}".format(np.mean(np.array(error))))
              with open('losses'+str(name)+'.txt','a') as loss_file:
                loss_file.write("avg difference between predicted and ground truth batch wise {}\n".format(np.mean(np.array(error))))
               
          if epoch% (hm_epochs-1)==0:
              error = []
              gt=[]
              predic=[]

              for i in range(int(2000/batch_size)):
                    batch_x, batch_y = next(next_test_batch)
                    pred = sess.run([logits], feed_dict={images: batch_x})
                    error.append(np.mean(abs(pred-batch_y)))
                    predic.append(pred)
                    gt.append(batch_y)

              print( "avg difference between predicted and ground truth batch wise {}".format(np.mean(np.array(error))))
              with open('losses'+str(name)+'.txt','a') as loss_file:
                loss_file.write("avg difference between predicted and ground truth batch wise {}\n".format(np.mean(np.array(error))))
               
            
              np.save(str(epoch)+'gt'+str(name)+'.npy',gt)
              np.save(str(epoch)+'predic'+str(name)+'.npy', predic)  

                
                
        
                
#           if epoch%999==0:
        

#           for i in range(10):

#                     batch_x, batch_y = next(next_test_batch)
#                     error=[]  
#                     pred = sess.run([logits], feed_dict={images: batch_x})
#                     error.append(np.mean(abs(pred-batch_y)))
#                     errors.append(error)
#                     predic.append(pred)
#                     gt.append(batch_y)
                    

                
          

#           np.savez(outfile,timing=timing,errors=errors,predic=predic,gt=gt)

        
          if epoch%10==0:
                save_path = saver.save(sess, os.path.join(ckpt_dir, "backup"+str(name)+str(epoch)))
          
      print('D')


In [7]:
train()

(?, 128, 128, 1)
(?, 64, 64, 32)
(?, 32, 32, 32)
(?, 16, 16, 64)
(?, 8, 8, 128)
(?, 4, 4, 256)
(?, 1)


C:\Users\Peyman\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 'U' mode is deprecated
  if sys.path[0] == '':


#Epoch 1 with total epochs 300 at step 0 loss: 65.18657684326172 running average of batch loss 65.18657684326172, time 8.856288433074951
#Epoch 1 with total epochs 300 at step 1 loss: 62.643409729003906 running average of batch loss 63.91499328613281, time 0.9326634407043457
#Epoch 1 with total epochs 300 at step 2 loss: 60.74011993408203 running average of batch loss 62.856702168782554, time 0.9526760578155518
#Epoch 1 with total epochs 300 at step 3 loss: 58.621009826660156 running average of batch loss 61.79777908325195, time 0.9556779861450195
#Epoch 1 with total epochs 300 at step 4 loss: 57.589412689208984 running average of batch loss 60.95610580444336, time 0.9676854610443115
#Epoch 1 with total epochs 300 at step 5 loss: 56.68901824951172 running average of batch loss 60.244924545288086, time 0.9576807022094727
#Epoch 1 with total epochs 300 at step 6 loss: 55.17842102050781 running average of batch loss 59.52113832746233, time 0.9397149085998535
#Epoch 1 with total epochs 300

#Epoch 3 with total epochs 300 at step 17 loss: 12.777122497558594 running average of batch loss 15.84757735994127, time 0.9776937961578369
#Epoch 3 with total epochs 300 at step 18 loss: 14.993975639343262 running average of batch loss 15.802650953594007, time 0.9506747722625732
#Epoch 3 with total epochs 300 at step 19 loss: 13.453082084655762 running average of batch loss 15.685172510147094, time 0.9606828689575195
#Epoch 3 with total epochs 300 at step 20 loss: 13.926064491271973 running average of batch loss 15.601405461629232, time 0.941666841506958
#Epoch 4 with total epochs 300 at step 0 loss: 13.30561637878418 running average of batch loss 13.30561637878418, time 0.9336633682250977
#Epoch 4 with total epochs 300 at step 1 loss: 13.096708297729492 running average of batch loss 13.201162338256836, time 0.9382033348083496
#Epoch 4 with total epochs 300 at step 2 loss: 12.690814018249512 running average of batch loss 13.031046231587728, time 0.9336621761322021
#Epoch 4 with total 

#Epoch 6 with total epochs 300 at step 13 loss: 11.131133079528809 running average of batch loss 11.482550689152308, time 0.9546761512756348
#Epoch 6 with total epochs 300 at step 14 loss: 12.366578102111816 running average of batch loss 11.541485850016276, time 0.9556798934936523
#Epoch 6 with total epochs 300 at step 15 loss: 12.435315132141113 running average of batch loss 11.597350180149078, time 0.9496500492095947
#Epoch 6 with total epochs 300 at step 16 loss: 11.136955261230469 running average of batch loss 11.570268126095042, time 0.9426701068878174
#Epoch 6 with total epochs 300 at step 17 loss: 11.522345542907715 running average of batch loss 11.567605760362413, time 0.9433948993682861
#Epoch 6 with total epochs 300 at step 18 loss: 12.544143676757812 running average of batch loss 11.619002492804276, time 0.9486727714538574
#Epoch 6 with total epochs 300 at step 19 loss: 11.301239013671875 running average of batch loss 11.603114318847656, time 0.9636836051940918
#Epoch 6 with

...

#Epoch 288 with total epochs 300 at step 7 loss: 3.3605456352233887 running average of batch loss 3.0079545378684998, time 0.49034881591796875
#Epoch 288 with total epochs 300 at step 8 loss: 2.9628148078918457 running average of batch loss 3.0029390123155384, time 0.5073332786560059
#Epoch 288 with total epochs 300 at step 9 loss: 3.143014907836914 running average of batch loss 3.0169466018676756, time 0.5123629570007324
#Epoch 288 with total epochs 300 at step 10 loss: 2.967212438583374 running average of batch loss 3.0124253142963755, time 0.5233728885650635
#Epoch 288 with total epochs 300 at step 11 loss: 2.862095594406128 running average of batch loss 2.999897837638855, time 0.4983501434326172
#Epoch 288 with total epochs 300 at step 12 loss: 2.990370035171509 running average of batch loss 2.9991649297567515, time 0.48864316940307617
#Epoch 288 with total epochs 300 at step 13 loss: 3.1230287551879883 running average of batch loss 3.0080123458589827, time 0.5008792877197266
#Epoc

#Epoch 291 with total epochs 300 at step 2 loss: 3.0241057872772217 running average of batch loss 3.1073683897654214, time 0.5293481349945068
#Epoch 291 with total epochs 300 at step 3 loss: 3.2560248374938965 running average of batch loss 3.1445325016975403, time 0.5083560943603516
#Epoch 291 with total epochs 300 at step 4 loss: 3.63885235786438 running average of batch loss 3.2433964729309084, time 0.5183446407318115
#Epoch 291 with total epochs 300 at step 5 loss: 3.438612222671509 running average of batch loss 3.2759324312210083, time 0.49035096168518066
#Epoch 291 with total epochs 300 at step 6 loss: 2.7693912982940674 running average of batch loss 3.2035694122314453, time 0.5123345851898193
#Epoch 291 with total epochs 300 at step 7 loss: 2.9085752964019775 running average of batch loss 3.166695147752762, time 0.5133669376373291
#Epoch 291 with total epochs 300 at step 8 loss: 3.392831325531006 running average of batch loss 3.1918213897281222, time 0.552391767501831
#Epoch 291 

#Epoch 293 with total epochs 300 at step 18 loss: 3.104048490524292 running average of batch loss 3.043927832653648, time 0.5093593597412109
#Epoch 293 with total epochs 300 at step 19 loss: 3.297903537750244 running average of batch loss 3.056626617908478, time 0.501356840133667
#Epoch 293 with total epochs 300 at step 20 loss: 3.075313091278076 running average of batch loss 3.0575164499736966, time 0.4883286952972412
#Epoch 294 with total epochs 300 at step 0 loss: 2.8986172676086426 running average of batch loss 2.8986172676086426, time 0.500330924987793
#Epoch 294 with total epochs 300 at step 1 loss: 3.083861827850342 running average of batch loss 2.991239547729492, time 0.5053579807281494
#Epoch 294 with total epochs 300 at step 2 loss: 3.1558127403259277 running average of batch loss 3.0460972785949707, time 0.49732518196105957
#Epoch 294 with total epochs 300 at step 3 loss: 2.9360268115997314 running average of batch loss 3.018579661846161, time 0.4893479347229004
#Epoch 294 w

#Epoch 296 with total epochs 300 at step 13 loss: 2.971745729446411 running average of batch loss 3.025551472391401, time 0.49134254455566406
#Epoch 296 with total epochs 300 at step 14 loss: 2.917851686477661 running average of batch loss 3.0183714866638183, time 0.49035000801086426
#Epoch 296 with total epochs 300 at step 15 loss: 3.1767802238464355 running average of batch loss 3.028272032737732, time 0.5433597564697266
#Epoch 296 with total epochs 300 at step 16 loss: 3.3358006477355957 running average of batch loss 3.046361951267018, time 0.5353829860687256
#Epoch 296 with total epochs 300 at step 17 loss: 3.026358127593994 running average of batch loss 3.0452506277296276, time 0.5183391571044922
#Epoch 296 with total epochs 300 at step 18 loss: 3.321506977081299 running average of batch loss 3.059790435590242, time 0.5233736038208008
#Epoch 296 with total epochs 300 at step 19 loss: 2.7928237915039062 running average of batch loss 3.0464421033859255, time 0.5013270378112793
#Epoc

#Epoch 299 with total epochs 300 at step 8 loss: 3.3255085945129395 running average of batch loss 3.025133000479804, time 0.509361982345581
#Epoch 299 with total epochs 300 at step 9 loss: 3.1684203147888184 running average of batch loss 3.0394617319107056, time 0.5353529453277588
#Epoch 299 with total epochs 300 at step 10 loss: 2.9183151721954346 running average of batch loss 3.0284484083002265, time 0.5123610496520996
#Epoch 299 with total epochs 300 at step 11 loss: 3.3531105518341064 running average of batch loss 3.0555035869280496, time 0.5033321380615234
#Epoch 299 with total epochs 300 at step 12 loss: 3.1253252029418945 running average of batch loss 3.060874480467576, time 0.5033309459686279
#Epoch 299 with total epochs 300 at step 13 loss: 3.043393135070801 running average of batch loss 3.059625812939235, time 0.5043303966522217
#Epoch 299 with total epochs 300 at step 14 loss: 3.4736461639404297 running average of batch loss 3.0872271696726483, time 0.4873473644256592
#Epoch

In [ ]:
x,y=next(next_test_batch)

In [ ]:
num=30

print(y[num])

In [ ]:
g=[]
for i in range(32):
    print(y[i])
    

In [ ]:
g=[]
for i in range(32):
    print(np.around(y[i],decimals=3))

In [ ]:
x=np.load('20gt_asp.npy')

In [ ]:
print(x)

In [ ]:
y=np.load('20predic_asp.npy')

In [ ]:
y.shape

In [ ]:
xx=np.reshape(x,(-1,1984))

In [ ]:
yy=np.reshape(y,(-1,1984))

In [ ]:
xx.shape

In [ ]:
plt.scatter(xx,yy)

In [4]:
name='world'

print('hello'+str(name))

helloworld
